In [ ]:
!pip install nptyping
!pip install thop

     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 133kB 22.1MB/s 


In [ ]:
!7z x fashion-mnist.7z
!7z x mnist.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 33072775 bytes (32 MiB)

Extracting archive: fashion-mnist.7z
--
Path = fashion-mnist.7z
Type = 7z
Physical Size = 33072775
Headers Size = 226
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     10% - fashion-mnist_test.csv                              18% 1 - fashion-mnist_train.csv                                 31% 1 - fashion-mnist_train.csv                                 43% 1 - fashion-mnist_train.csv                                 53% 1 - fashion-mnist_tra

In [ ]:
# torch
import torch
import torch.nn       as     nn
import torch.optim    as     optim
from   torch.autograd import Function

# torch utils
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# additional libraries
from typing import Any, Iterable, MutableSequence, Tuple, Text, Tuple, List, Mapping, Sequence, Union, Dict
import os
import urllib.request
import zipfile
import time
import math
import numpy             as np
import matplotlib.pyplot as plt
import pandas as pd
from thop import profile, clever_format
from tqdm.notebook import tqdm
from nptyping import NDArray
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
# from skorch import NeuralNetClassifier, NeuralNet
from itertools import product
%matplotlib inline

In [ ]:
hyper_parameters: Mapping[
    str,
    Sequence[Union[int, float]]
] = {
    "n_linear_warmup_epochs": [1, 3, 5],
    "learning_rate_max": [0.01, 0.001, 0.0001],
}
# fine_hyper_parameters: Mapping[
#     str,
#     Mapping[Union[int, float], Sequence[Union[int, float]]]
# ] = {
#     "n_linear_warmup_epochs": {
#         1: [0, 1, 2],
#         3: [2, 3, 4],
#         5: [4, 5, 6],
#     },
#     "learning_rate_max": {
#         0.01: [0.055, 0.01, 0.0055],
#         0.001: [0.0055, 0.001, 0.00055],
#         0.0001: [0.00055, 0.0001, 0.000055],
#     },
# }

# Functions

In [ ]:
def create_numpy_data(file: Text) -> Tuple[NDArray,NDArray]:
    cvs_data = pd.read_csv(file)
    return (
        np.reshape(
            np.array(
                cvs_data.iloc[:,1:],
                dtype=np.int32
            ),
            (
                len(cvs_data.index),1,28,28
            )
        ),
        np.array(
            cvs_data.iloc[:,0],
            dtype=np.int32
        )
    )

In [ ]:
def create_torch_dataloader(
    training_data_0: NDArray, training_label_0: NDArray,
    validation_data_0: NDArray, validation_label_0: NDArray,
    test_data_0: NDArray, test_label_0: NDArray,
    
    training_data_1: NDArray, training_label_1: NDArray,
    validation_data_1: NDArray, validation_label_1: NDArray,
    test_data_1: NDArray, test_label_1: NDArray,

    batch_size: int = 32
) -> Tuple[
           torch.utils.data.DataLoader,
           torch.utils.data.DataLoader,
           torch.utils.data.DataLoader
    ]:
    

    X_train = torch.from_numpy(
        np.concatenate(
            (training_data_0,training_data_1)
        )
    ).float()
    X_validation = torch.from_numpy(
        np.concatenate(
            (validation_data_0,validation_data_1)
        )
    ).float()
    X_test = torch.from_numpy(
        np.concatenate(
            (test_data_0,test_data_1)
        )
    ).float()

    # concept - 0
    modified_labels_training_0 = np.zeros(len(training_label_0))
    modified_labels_validation_0 = np.zeros(len(validation_label_0))
    modified_labels_testing_0 = np.zeros(len(test_label_0))
    # tangible - 1
    modified_labels_training_1 = np.ones(len(training_label_1))
    modified_labels_validation_1 = np.ones(len(validation_label_1))
    modified_labels_testing_1 = np.ones(len(test_label_1))

    y_train = torch.from_numpy(
        np.concatenate(
            (modified_labels_training_0,modified_labels_training_1)
        )
    ).long()
    y_validation = torch.from_numpy(
        np.concatenate(
            (modified_labels_validation_0,modified_labels_validation_1)
        )
    ).long()
    y_test = torch.from_numpy(
        np.concatenate(
            (modified_labels_testing_0,modified_labels_testing_1)
        )
    ).long()

    print('training data shape: {shape}'.format(shape=X_train.shape))
    print('validation data shape: {shape}'.format(shape=X_validation.shape))
    print('test data shape: {shape}'.format(shape=X_test.shape))
    print('training labels shape: {shape}'.format(shape=y_train.shape))
    print('validation labels shape: {shape}'.format(shape=y_validation.shape))
    print('test labels shape: {shape}'.format(shape=y_test.shape))

    transform_train = transforms.Compose(
        [
         transforms.RandomCrop((20, 20)),
         transforms.Pad(4),
         transforms.Normalize((0.5),(0.5))
        ]
    )
    transform_validation_test = transforms.Compose(
        [
         transforms.CenterCrop((24, 24)),
         transforms.Pad(2),
         transforms.Normalize((0.5),(0.5))
        ]
    )

    return (
        torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(
                transform_train(X_train),y_train
            ),
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            drop_last=True
        ),
        torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(
                transform_validation_test(X_validation),y_validation
            ),
            batch_size=batch_size,
            shuffle=False,
            num_workers=4,
            drop_last=True
        ),
        torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(
                transform_validation_test(X_test),y_test
            ),
            batch_size=batch_size,
            shuffle=False,
            num_workers=4,
            drop_last=True
        )
    )

In [ ]:
def hyper_train(
    model: nn.Module,
    dataloader_train: torch.utils.data.DataLoader,
    dataloader_val: torch.utils.data.DataLoader,
    dataloader_test: torch.utils.data.DataLoader,
    file_name: Text,
    file_save: int,
    file_load: int,
    max_epochs: int
) -> List[
          Tuple[
                Dict[
                     str,
                     Union[
                           int,
                           float
                          ]
                    ],
                List[
                     Tuple[
                           float,
                           float,
                           float
                    ]
                ]
        ]
    ]:
    hyper_param_training_stats = []
    for params in product(*map(hyper_parameters.get,list(hyper_parameters))):
        params_dict = dict(zip(list(hyper_parameters),params))
        print(params_dict)
        training_stats = train(
            model,
            dataloader_train,
            dataloader_val,
            dataloader_test,
            file_name,
            file_save,
            file_load,
            params_dict,
            max_epochs
        )
        hyper_param_training_stats.append((params_dict,training_stats))
    
    print(np.array(hyper_param_training_stats))
    return (hyper_param_training_stats)

In [ ]:
def train(
    model: nn.Module,
    dataloader_train: torch.utils.data.DataLoader,
    dataloader_val: torch.utils.data.DataLoader,
    dataloader_test: torch.utils.data.DataLoader,
    file_name: Text,
    file_save: int,
    file_load: int,
    params: Dict[str,Union[int,float]],
    max_epochs: int
) -> List[Tuple[float,float,float]]:
    assert params['n_linear_warmup_epochs'] < max_epochs, 'initial epochs should be less than max epochs'

    # training (linear warm up with cosine decay learning rate)
    TRAINING_LR_MAX = params['learning_rate_max']
    TRAINING_LR_INIT_SCALE   = 0.01
    TRAINING_LR_INIT_EPOCHS  = params['n_linear_warmup_epochs']
    TRAINING_LR_FINAL_SCALE  = 0.01
    TRAINING_LR_FINAL_EPOCHS = max_epochs - TRAINING_LR_INIT_EPOCHS
    TRAINING_NUM_EPOCHS      = TRAINING_LR_INIT_EPOCHS + TRAINING_LR_FINAL_EPOCHS
    TRAINING_LR_INIT         = TRAINING_LR_MAX*TRAINING_LR_INIT_SCALE
    TRAINING_LR_FINAL        = TRAINING_LR_MAX*TRAINING_LR_FINAL_SCALE
    # start epoch
    start_epoch = 0
    # print('initial_epochs: ',TRAINING_LR_INIT_EPOCHS)
    # print('final_epochs: ',TRAINING_LR_FINAL_EPOCHS)
    # print('total_epochs: ',TRAINING_NUM_EPOCHS)
    # print('max_learning_rate: ',TRAINING_LR_MAX)
    # print('init_learning_rate: ',TRAINING_LR_INIT)
    # print('final_learning_rate: ',TRAINING_LR_FINAL)
    
    # learning rate schedule
    def lr_schedule(epoch):
        # linear warmup followed by cosine decay
        if epoch < TRAINING_LR_INIT_EPOCHS:
            lr = (TRAINING_LR_MAX - TRAINING_LR_INIT)*(float(epoch)/TRAINING_LR_INIT_EPOCHS) + TRAINING_LR_INIT
        else:
            lr = (TRAINING_LR_MAX - TRAINING_LR_FINAL)*max(0.0, math.cos(((float(epoch) - TRAINING_LR_INIT_EPOCHS)/(TRAINING_LR_FINAL_EPOCHS - 1.0))*(math.pi/2.0))) + TRAINING_LR_FINAL
        return lr

    # error (softmax cross entropy)
    criterion = nn.CrossEntropyLoss()

    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    # specify the device as the GPU if present with fallback to the CPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # print(device)

    # transfer the network to the device
    model.to(device)

    # model loading
    if file_load == 1:
        checkpoint = torch.load(file_name)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1

    # cycle through the epochs
    training_statistics: List[Tuple[float,float]] = []
    for epoch in range(start_epoch, TRAINING_NUM_EPOCHS):

        # initialize train set statistics
        model.train()
        training_loss = 0.0
        num_batches   = 0

        # set the learning rate for the epoch
        for g in optimizer.param_groups:
            g['lr'] = lr_schedule(epoch)

        # cycle through the train set
        for data in tqdm(dataloader_train):

            # extract a batch of data and move it to the appropriate device
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward pass, loss, backward pass and weight update
            outputs = model(inputs)
            loss    = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # update statistics
            training_loss = training_loss + loss.item()
            num_batches   = num_batches + 1

        # initialize test set statistics
        model.eval()
        test_correct = 0
        test_total   = 0

        val_loss = 0.0
        val_num_batches   = 0
        # no weight update / no gradient needed
        with torch.no_grad():

            # cycle through the test set
            for data in tqdm(dataloader_val):

                # extract a batch of data and move it to the appropriate device
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                # forward pass and prediction
                outputs      = model(inputs)
                predicted = torch.max(outputs.data, 1)[1]

                # loss    = criterion(outputs, labels)
                # loss.backward()
                # optimizer.step()

                # update test set statistics
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
                val_loss += criterion(outputs, labels).item()
                val_num_batches += 1


        # epoch statistics
        print('Epoch {0:2d} lr = {1:8.6f} avg loss = {2:8.6f} avg validation loss = {3:8.6f} accuracy = {4:5.2f}'.format(
            epoch,
            lr_schedule(epoch),
            (training_loss/num_batches)/32,
            (val_loss/val_num_batches)/32,
            (100.0*test_correct/test_total)
            )
        )
        #print(f"Epoch {epoch} lr = {lr_schedule(epoch)} avg train loss = {np.float32((training_loss/num_batches)/32)}  avg val loss = {np.float32(val_loss/val_num_batches/32)}")
        training_statistics.append(
            (
                (training_loss/num_batches)/32,
                (val_loss/val_num_batches)/32,
                (100.0*test_correct/test_total)
            )
        )

    # model saving
    # to use this for checkpointing put this code block inside the training loop at the end (e.g., just indent it 4 spaces)
    # and set 'epoch' to the current epoch instead of TRAINING_NUM_EPOCHS - 1; then if there's a crash it will be possible
    # to load this checkpoint and restart training from the last complete epoch instead of having to start training at the
    # beginning
        if file_save == 1:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, file_name)

    return training_statistics

# Pre-Processing

In [ ]:
mnist_data_train, mnist_labels_train = create_numpy_data('./mnist_train.csv')
mnist_data_val, mnist_labels_val = create_numpy_data('./mnist_val.csv')
mnist_data_test, mnist_labels_test = create_numpy_data('./mnist_test.csv')
fashion_data_train, fashion_labels_train = create_numpy_data('./fashion-mnist_train.csv')
fashion_data_val, fashion_labels_val = create_numpy_data('./fashion-mnist_val.csv')
fashion_data_test, fashion_labels_test = create_numpy_data('./fashion-mnist_test.csv')

In [ ]:
(dataloader_train, dataloader_val, dataloader_test) = create_torch_dataloader(
        mnist_data_train, mnist_labels_train,
        mnist_data_val, mnist_labels_val,
        mnist_data_test, mnist_labels_test,
        fashion_data_train, fashion_labels_train,
        fashion_data_val, fashion_labels_val,
        fashion_data_test, fashion_labels_test
)

training data shape: torch.Size([98000, 1, 28, 28])
validation data shape: torch.Size([14000, 1, 28, 28])
test data shape: torch.Size([28000, 1, 28, 28])
training labels shape: torch.Size([98000])
validation labels shape: torch.Size([14000])
test labels shape: torch.Size([28000])


# ResNetV2

In [ ]:
# ResNetV2 bottleneck
class ResNetV2Bottleneck(nn.Module):

    # initialization
    def __init__(self, in_channels, residual_channels, out_channels, stride_num):

        # parent initialization
        super(ResNetV2Bottleneck, self).__init__()

        # identity
        if ((in_channels != out_channels) or (stride_num > 1)):
            self.conv0_present = True
            self.conv0         = nn.Conv2d(in_channels, out_channels, (1, 1), stride=(stride_num, stride_num), padding=(0, 0), dilation=(1, 1), groups=1, bias=False, padding_mode='zeros')
        else:
            self.conv0_present = False

        # residual
        self.block_1 = nn.ModuleList()
        self.block_1.append(nn.BatchNorm2d(in_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.block_1.append(nn.ReLU())
        self.block_1.append(nn.Conv2d(in_channels, residual_channels, (1, 1), stride=(1, 1), padding=(0, 0), dilation=(1, 1), groups=1, bias=False, padding_mode='zeros'))
        
        self.block_2 = nn.ModuleList()
        self.block_2.append(nn.BatchNorm2d(residual_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.block_2.append(nn.ReLU())
        self.block_2.append(nn.Conv2d(residual_channels, residual_channels, (3, 3), stride=(stride_num, stride_num), padding=(1, 1), dilation=(1, 1), groups=1, bias=False, padding_mode='zeros'))

        self.block_3 = nn.ModuleList()
        self.block_3.append(nn.BatchNorm2d(residual_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.block_3.append(nn.ReLU())
        self.block_3.append(nn.Conv2d(residual_channels, out_channels, (1, 1), stride=(1, 1), padding=(0, 0), dilation=(1, 1), groups=1, bias=False, padding_mode='zeros'))

    # forward path
    def forward(self, x):

        res = x
        # residual
        for layer in self.block_1:
            res = layer(res)

        for layer in self.block_2:
            res = layer(res)

        for layer in self.block_3:
            res = layer(res)

        # identity
        if (self.conv0_present == True):
            x = self.conv0(x)

        # summation
        x = x + res

        # return
        return x

# ResNetV2 model
class ResNetModel(nn.Module):
    # initialization
    def __init__(self):

        # parent initialization
        super(ResNetModel, self).__init__()

        # encoder tail
        self.enc_tail = nn.ModuleList()
        self.enc_tail.append(nn.Conv2d(1, 32, (3, 3), stride=(1, 1), padding=(3, 3), dilation=(1, 1), groups=1, bias=False, padding_mode='zeros'))


        # encoder level 0
        self.enc_0 = nn.ModuleList()
        self.enc_0.append(ResNetV2Bottleneck(32, 1 * 32, 128, 1))
        for n in range(4 - 1):
            self.enc_0.append(ResNetV2Bottleneck(128, 1 * 32, 128, 1))

        # encoder level 1
        self.enc_1 = nn.ModuleList()
        self.enc_1.append(ResNetV2Bottleneck(128,64, 256, 2))
        for n in range(6 - 1):
            self.enc_1.append(ResNetV2Bottleneck(256,64, 256, 1))

        # encoder level 2
        self.enc_2 = nn.ModuleList()
        self.enc_2.append(ResNetV2Bottleneck(256, 128, 512, 2))
        for n in range(3 - 1):
            self.enc_2.append(ResNetV2Bottleneck(512, 128, 512, 1))

        # encoder level 3
        self.enc_3 = nn.ModuleList()
        self.enc_3.append(nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.enc_3.append(nn.ReLU())

        # decoder
        self.dec = nn.ModuleList()
        self.dec.append(nn.AdaptiveAvgPool2d((1, 1)))
        self.dec.append(nn.Flatten())
        self.dec.append(nn.Linear(512,2,bias=True))
        # data_num_classes = 2 for concept and intangable
        

    # forward path
    def forward(self, x):

        # encoder tail
        for layer in self.enc_tail:
            x = layer(x)
        
        # encoder level 0
        for layer in self.enc_0:
            x = layer(x)

        # encoder level 1
        for layer in self.enc_1:
            x = layer(x)

        # encoder level 2
        for layer in self.enc_2:
            x = layer(x)

        # encoder level 3 
        for layer in self.enc_3:
            x = layer(x)

        # decoder
        for layer in self.dec:
            x = layer(x)

        # return
        return x

resnet_model = ResNetModel()
print(resnet_model)

ResNetModel(
  (enc_tail): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
  )
  (enc_0): ModuleList(
    (0): ResNetV2Bottleneck(
      (conv0): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (block_1): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (block_2): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (block_3): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): ResNetV2Bottleneck(
      (block_1

In [ ]:
ResNetV2_training_statistics: List[Tuple[float,float]] = train(
    dataloader_train,
    dataloader_test,
    'ResNetV2.pt',
    1,
    0
)


Epoch  0 lr = 0.000010 avg loss = 0.063430 accuracy = 50.14



Epoch  1 lr = 0.000208 avg loss = 0.015888 accuracy = 66.24



Epoch  2 lr = 0.000406 avg loss = 0.009466 accuracy = 73.49



Epoch  3 lr = 0.000604 avg loss = 0.007829 accuracy = 75.97



Epoch  4 lr = 0.000802 avg loss = 0.007246 accuracy = 78.24



Epoch  5 lr = 0.001000 avg loss = 0.006827 accuracy = 82.02



Epoch  6 lr = 0.000925 avg loss = 0.005968 accuracy = 78.94



Epoch  7 lr = 0.000710 avg loss = 0.005121 accuracy = 80.06



Epoch  8 lr = 0.000389 avg loss = 0.004109 accuracy = 78.84



Epoch  9 lr = 0.000010 avg loss = 0.003203 accuracy = 83.58


In [ ]:
size = torch.randn(1,1,28,28)
macs, params = profile(model, inputs=(size, ))
print(macs, params)
macs, params = clever_format([macs, params], "%.3f")
print(macs, params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class '__main__.ResNetV2Bottleneck'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.flatten.Flatten'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.ResNetModel'>. Treat it as zero Macs and zero Params.
523097088.0 1468276.0
523.097M 1.468M


In [ ]:
resnet_hyper_stats = hyper_train(
    resnet_model,
    dataloader_train,
    dataloader_val,
    dataloader_test,
    'resnet.pt',
    1,
    0,
    10
)

{'n_linear_warmup_epochs': 1, 'learning_rate_max': 0.01}



Epoch  0 lr = 0.000100 avg loss = 0.000002 avg validation loss = 0.000080 accuracy = 99.91



Epoch  1 lr = 0.010000 avg loss = 0.000156 avg validation loss = 0.000181 accuracy = 99.80



Epoch  2 lr = 0.009810 avg loss = 0.000090 avg validation loss = 0.002223 accuracy = 98.17



Epoch  3 lr = 0.009246 avg loss = 0.000066 avg validation loss = 0.000029 accuracy = 99.96


...


Epoch  5 lr = 0.000071 avg loss = 0.000000 avg validation loss = 0.000038 accuracy = 99.94



Epoch  6 lr = 0.000056 avg loss = 0.000000 avg validation loss = 0.000036 accuracy = 99.96



Epoch  7 lr = 0.000039 avg loss = 0.000000 avg validation loss = 0.000091 accuracy = 99.89



Epoch  8 lr = 0.000020 avg loss = 0.000000 avg validation loss = 0.000047 accuracy = 99.93


Buffered data was truncated after reaching the output size limit.

In [ ]:
print(np.array(resnet_hyper_stats))

[[{'n_linear_warmup_epochs': 1, 'learning_rate_max': 0.01}
  list([(1.6249627904496623e-06, 8.02116079524917e-05, 99.90703661327231), (0.00015584523370013072, 0.0001808866674389847, 99.79977116704805), (8.988849901356172e-05, 0.0022225601288794598, 98.16933638443936), (6.569688863364445e-05, 2.9170887523465176e-05, 99.96424485125858), (4.660688706793767e-05, 0.0001638179384106271, 99.8283752860412), (3.643865061459333e-05, 0.00013259366833515758, 99.86413043478261), (2.5745052611784665e-05, 0.00017924573374469767, 99.79977116704805), (1.5785616738316144e-05, 0.004145023771662938, 96.38872997711671), (4.7804709608873465e-06, 0.0002753011185061759, 99.64244851258582), (2.5043089486809575e-06, 0.00013988538419819072, 99.84982837528604)])]
 [{'n_linear_warmup_epochs': 1, 'learning_rate_max': 0.001}
  list([(1.7769004129485866e-06, 8.56011257019023e-05, 99.89988558352402), (2.8164315579944208e-06, 9.921605574769304e-05, 99.86413043478261), (9.913994820934125e-07, 7.070497543209516e-05, 99.9

# MobileNetV2

In [ ]:
class MobileNetV2ModelBottleneck(nn.Module):
    def __init__(
        self,
        input_channels: int,
        output_channels: int,
        stride: int,
        expansion_multiplier: int,
    ):
        super(MobileNetV2ModelBottleneck, self).__init__()

        self.use_residual_connection: bool = (
            stride == 1 and input_channels == output_channels
        )

        expansion_channels: int = int(round(input_channels * expansion_multiplier))

        convolution_layers: MutableSequence[nn.Module] = (
            [
                # 1x1 Kernel Projection-Wise Linear Convolution.
                # Note: We do this to make sure that before we do depth-wise
                # convolution the input channel size is the same as the
                # expansion channel size
                nn.Conv2d(input_channels, expansion_channels, 1, 1, 0, bias=False),
                nn.BatchNorm2d(expansion_channels),
                nn.ReLU6(inplace=True),
            ]
            if expansion_multiplier != 1
            else []
        )
        convolution_layers.extend(
            [
                # Depth-Wise Convolution
                nn.Conv2d(
                    expansion_channels,
                    expansion_channels,
                    3,
                    stride,
                    1,
                    groups=expansion_channels,
                    bias=False,
                ),
                nn.BatchNorm2d(expansion_channels),
                nn.ReLU6(inplace=True),
                # 1x1 Kernel Projection-Wise Linear Convolution
                nn.Conv2d(expansion_channels, output_channels, 1, 1, 0, bias=False),
                nn.BatchNorm2d(output_channels),
            ]
        )

        self.convolution_layers = nn.Sequential(*convolution_layers)

    def forward(self, x: Any) -> Any:
        if self.use_residual_connection:
            return x + self.convolution_layers(x)
        else:
            return self.convolution_layers(x)


class MobileNetV2Model(nn.Module):
    def __init__(self, num_classes: int = 2, width_mult: float = 1.0):
        super(MobileNetV2Model, self).__init__()

        # Setup Input (Tail Section). The first layer should be a 3x3
        # convolution into batch norm into an ReLU6 activation function
        input_channel: int = 32
        layers: MutableSequence[nn.Module] = [
            nn.Sequential(
                nn.Conv2d(1, input_channel, 3, 2, 1, bias=False),
                nn.BatchNorm2d(input_channel),
                nn.ReLU6(inplace=True),
            )
        ]

        # Build Inverted Residual Bottleneck Layers (Body Section)
        body_hyperparameters: Iterable[Tuple[int, int, int, int]] = [
            (1, 16, 1, 1),
            (6, 24, 2, 2),
            (6, 32, 3, 2),
            (6, 64, 4, 2),
            (6, 96, 3, 1),
            (6, 160, 3, 2),
            (6, 320, 1, 1),
        ]

        for (
            expansion_multiplier,
            output_channels,
            num_repeats,
            stride,
        ) in body_hyperparameters:
            if expansion_multiplier > 1:
                output_channels = make_divisible(output_channels * width_mult)
            for i in range(1, num_repeats):
                layers.append(
                    MobileNetV2ModelBottleneck(
                        input_channel,
                        output_channels,
                        stride if i == 0 else 1,
                        expansion_multiplier,
                    )
                )
                input_channel = output_channels

        # Add last layer in body section. Combine Tail and Body sections together
        last_layer_channels: int = (
            make_divisible(1280 * width_mult) if width_mult > 1.0 else 1280
        )
        self.features = nn.Sequential(
            *layers,
            nn.Sequential(
                nn.Conv2d(input_channel, last_layer_channels, 1, 1, 0, bias=False),
                nn.BatchNorm2d(last_layer_channels),
                nn.ReLU6(inplace=True),
            ),
        )

        # Build Head Section (classifier)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(last_layer_channels,num_classes)
        )

        # Initilize Weights of the Layers
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x: Any) -> Any:
        x = self.features(x)
        return self.classifier(x)


mobilenet_v2_model = MobileNetV2Model()
print(mobilenet_v2_model)

NameError: ignored

In [ ]:
MobileNetV2_training_statistics: List[Tuple[float,float]] = train(
    dataloader_train,
    dataloader_test,
    'MobileNetV2.pt',
    1,
    0
)


Epoch  0 lr = 0.000010 avg loss = 0.011045 accuracy = 58.94



Epoch  1 lr = 0.000208 avg loss = 0.012826 accuracy = 66.12



Epoch  2 lr = 0.000406 avg loss = 0.010235 accuracy = 76.19



Epoch  3 lr = 0.000604 avg loss = 0.009198 accuracy = 68.50



Epoch  4 lr = 0.000802 avg loss = 0.008315 accuracy = 73.04



Epoch  5 lr = 0.001000 avg loss = 0.008146 accuracy = 79.81



Epoch  6 lr = 0.000925 avg loss = 0.006507 accuracy = 76.56



Epoch  7 lr = 0.000710 avg loss = 0.005149 accuracy = 82.08



Epoch  8 lr = 0.000389 avg loss = 0.003691 accuracy = 83.33



Epoch  9 lr = 0.000010 avg loss = 0.002440 accuracy = 84.39


# MobileNetV3 (Small)

In [ ]:
def make_divisible(dividend: float, divisor: int = 8) -> int:
    # It was used in Google's implementation so we decided to copy it over
    return int(np.ceil(dividend * 1.0 / divisor) * divisor)


class HSigmoid(nn.Module):
    def __init__(self, inplace: bool = True):
        super(HSigmoid, self).__init__()
        self.relu = nn.ReLU6(inplace=inplace)

    def forward(self, x: float) -> float:
        return self.relu(x + 3) / 6


class HSwish(nn.Module):
    def __init__(self, inplace: bool = True):
        super(HSwish, self).__init__()
        self.sigmoid = HSigmoid(inplace=inplace)

    def forward(self, x: float) -> float:
        return x * self.sigmoid(x)


class SELayer(nn.Module):
    def __init__(self, channel: int, reduction: int = 4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fully_connected_layer = nn.Sequential(
            nn.Linear(channel, make_divisible(channel // reduction)),
            nn.ReLU(inplace=True),
            nn.Linear(make_divisible(channel // reduction), channel),
            HSigmoid(),
        )

    def forward(self, x: Any) -> Any:
        b, c = x.size()[:2]
        y = self.avg_pool(x).view(b, c)
        y = self.fully_connected_layer(y).view(b, c, 1, 1)
        return x * y


class InvertedResidualBottleneck(nn.Module):
    def __init__(
        self,
        input_channels: int,
        expansion_channels: int,
        oup: int,
        kernel_size: int,
        stride: int,
        use_se: bool,
        activation_function_name: str,
    ):
        super(InvertedResidualBottleneck, self).__init__()

        self.use_residual_connection: bool = stride == 1 and input_channels == oup

        squeeze_excite_layer: nn.Module = (
            SELayer(expansion_channels) if use_se else nn.Identity()
        )

        activation_function: nn.Module = (
            nn.ReLU(inplace=True) if activation_function_name == "ReLU" else HSwish()
        )

        convolution_layers: MutableSequence[nn.Module] = (
            [
                # 1x1 Kernel Projection-Wise Linear Convolution.
                # Note: We do this to make sure that before we do depth-wise
                # convolution the input channel size is the same as the
                # expansion channel size
                nn.Conv2d(input_channels, expansion_channels, 1, 1, 0, bias=False),
                nn.BatchNorm2d(expansion_channels),
                activation_function,
            ]
            if input_channels != expansion_channels
            else []
        )
        convolution_layers.extend(
            [
                # Depth-Wise Convolution
                nn.Conv2d(
                    expansion_channels,
                    expansion_channels,
                    kernel_size,
                    stride,
                    (kernel_size - 1) // 2,
                    groups=expansion_channels,
                    bias=False,
                ),
                nn.BatchNorm2d(expansion_channels),
                # Squeeze-Excite Layer
                squeeze_excite_layer,
                activation_function,
                # 1x1 Kernel Projection-Wise Linear Convolution
                nn.Conv2d(expansion_channels, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            ]
        )

        self.convolution_layers = nn.Sequential(*convolution_layers)


    def forward(self, x: Any) -> Any:
        if self.use_residual_connection:
            return x + self.convolution_layers(x)
        else:
            return self.convolution_layers(x)


class MobileNetV3Small(nn.Module):
    def __init__(self, num_classes: int = 2, width_mult: float = 1.0):
        super(MobileNetV3Small, self).__init__()

        # Setup Input (Tail Section). The first layer should be a 3x3
        # convolution into batch norm into an H-swish activation function
        input_channel: int = make_divisible(1 * width_mult)
        # input_channel: int = 16
        layers: MutableSequence[nn.Module] = [
            nn.Sequential(
                nn.Conv2d(1, input_channel, 3, 2, 1, bias=False),
                nn.BatchNorm2d(input_channel),
                HSwish(),
            )
        ]

        # Build Inverted Residual Bottleneck Layers (Body Section)
        body_hyperparameters: Iterable[Tuple[int, int, int, bool, str, int]] = [
            (3, 16, 16, True, "ReLU", 2),
            (3, 72, 24, False, "ReLU", 2),
            (3, 88, 24, False, "ReLU", 1),
            (5, 96, 40, True, "H-Swish", 2),
            (5, 240, 40, True, "H-Swish", 1),
            (5, 240, 40, True, "H-Swish", 1),
            (5, 120, 48, True, "H-Swish", 1),
            (5, 144, 48, True, "H-Swish", 1),
            (5, 288, 96, True, "H-Swish", 2),
            (5, 576, 96, True, "H-Swish", 1),
            (5, 576, 96, True, "H-Swish", 1),
        ]

        for (
            kernel_size,
            expansion_size,
            output_channels,
            use_squeeze_excitation,
            activation_function,
            stride,
        ) in body_hyperparameters:
            output_channels = make_divisible(output_channels * width_mult)
            expansion_channels = make_divisible(expansion_size * width_mult)
            layers.append(
                InvertedResidualBottleneck(
                    input_channel,
                    expansion_channels,
                    output_channels,
                    kernel_size,
                    stride,
                    use_squeeze_excitation,
                    activation_function,
                )
            )
            input_channel = output_channels

        # Combine Tail and Body sections together
        self.features = nn.Sequential(*layers)

        # Build Head Section
        head_output_size: int = make_divisible(960 * width_mult)
        self.conv = nn.Sequential(
            nn.Conv2d(input_channel, head_output_size, 1, 1, 0, bias=False),
            nn.BatchNorm2d(head_output_size),
            HSwish(),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        output_channel: int = (
            make_divisible(1024 * width_mult) if width_mult > 1.0 else 1024
        )
        self.classifier = nn.Sequential(
            nn.Linear(head_output_size, output_channel),
            HSwish(),
            nn.Dropout(0.2),
            nn.Linear(output_channel, num_classes),
        )


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x: Any) -> Any:
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

mobilenetV3_model = MobileNetV3Small()
print(mobilenetV3_model)

MobileNetV3Small(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): HSwish(
        (sigmoid): HSigmoid(
          (relu): ReLU6(inplace=True)
        )
      )
    )
    (1): InvertedResidualBottleneck(
      (convolution_layers): Sequential(
        (0): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fully_connected_layer): Sequential(
            (0): Linear(in_features=16, out_features=

In [ ]:
MobileNetV3_training_statistics: List[Tuple[float,float]] = train(
    dataloader_train,
    dataloader_test,
    'MobileNetV3.pt',
    1,
    0,
)


Epoch  0 lr = 0.000010 avg loss = 0.000022



Epoch  1 lr = 0.000208 avg loss = 0.000044



Epoch  2 lr = 0.000406 avg loss = 0.000082



Epoch  3 lr = 0.000604 avg loss = 0.000137



Epoch  4 lr = 0.000802 avg loss = 0.000137


...


Epoch 12 lr = 0.000545 avg loss = 0.000019



Epoch 13 lr = 0.000421 avg loss = 0.000006



Epoch 14 lr = 0.000289 avg loss = 0.000005



Epoch 15 lr = 0.000151 avg loss = 0.000004



Epoch 16 lr = 0.000010 avg loss = 0.000000


In [ ]:
print(np.array(MobileNetV3_training_statistics))

[[2.24087820e-05 9.93428571e+01]
 [4.39969041e-05 9.90714286e+01]
 [8.24071365e-05 9.79250000e+01]
 [1.36589787e-04 9.84642857e+01]
 [1.36772134e-04 9.78357143e+01]
 [1.21514813e-04 9.64071429e+01]
 [2.26394632e-04 9.85178571e+01]
 [9.95440522e-05 9.76285714e+01]
 [1.15148404e-04 9.85785714e+01]
 [4.57348991e-05 9.87821429e+01]
 [4.00231930e-05 9.68571429e+01]
 [6.11422220e-05 9.86071429e+01]
 [1.85979983e-05 9.87107143e+01]
 [5.97901079e-06 9.87964286e+01]
 [4.72982166e-06 9.82857143e+01]
 [3.77712506e-06 9.85571429e+01]
 [1.64652385e-07 9.86321429e+01]]


# Results

In [ ]:
macs, params = profile(model, inputs=(size, ))
print(macs, params)
macs, params = clever_format([macs, params], "%.3f")
print(macs, params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[WARN] Cannot find rule for <class '__main__.HSigmoid'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.HSwish'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.SELayer'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.InvertedResidualBottleneck'>. Treat it as zero Macs and zero Params.
[WARN]

In [ ]:
print(classification_report(flatten_y_true,flatten_y_pred)) # MobileNet V3

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     14000
           1       0.99      0.98      0.99     14000

    accuracy                           0.99     28000
   macro avg       0.99      0.99      0.99     28000
weighted avg       0.99      0.99      0.99     28000



In [ ]:
print(confusion_matrix(flatten_y_true,flatten_y_pred)) #MOBILENet V3

[[13851   149]
 [  234 13766]]


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# no weight update / no gradient needed
y_pred = []
y_true = []
with torch.no_grad():

    # cycle through the test set
    for data in dataloader_test:

        # extract a batch of data and move it to the appropriate device
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # forward pass and prediction
        outputs      = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted.cpu().numpy())
        y_true.append(labels.cpu().numpy())

y_pred = [a.squeeze().tolist() for a in y_pred]
y_true = [a.squeeze().tolist() for a in y_true]

In [ ]:
flatten_y_pred = []
for sublist in y_pred:
    for x in sublist:
        flatten_y_pred.append(x)
flatten_y_true = []
for sublist in y_true:
    for x in sublist:
        flatten_y_true.append(x)

In [ ]:
print(classification_report(flatten_y_true,flatten_y_pred))
mobilenetV3_report = classification_report(flatten_y_true,flatten_y_pred, output_dict=True)
df = pd.DataFrame(mobilenetV3_report).transpose()
df.to_csv('mobilenetV3_report.csv',index=True)

              precision    recall  f1-score   support

           0       0.94      0.25      0.40      1381
           1       0.52      0.94      0.67      1575
           2       0.67      0.92      0.77      1398
           3       0.71      0.73      0.72      1428
           4       0.89      0.81      0.85      1365
           5       0.84      0.79      0.81      1263
           6       0.88      0.85      0.87      1375
           7       0.94      0.85      0.89      1459
           8       0.69      0.61      0.65      1365
           9       0.85      0.83      0.84      1391
          10       0.85      0.53      0.65      1400
          11       0.90      0.87      0.88      1400
          12       0.56      0.34      0.43      1400
          13       0.60      0.48      0.53      1400
          14       0.57      0.62      0.59      1400
          15       0.69      0.78      0.73      1400
          16       0.38      0.52      0.44      1400
          17       0.75    

In [ ]:
print(classification_report(flatten_y_true,flatten_y_pred)) #RESNET V2
resnetV2_report = classification_report(flatten_y_true,flatten_y_pred, output_dict=True)
df = pd.DataFrame(resnetV2_report).transpose()
df.to_csv('resnetV2_report.csv',index=True)

              precision    recall  f1-score   support

           0       0.94      0.73      0.82      1381
           1       0.82      0.87      0.84      1575
           2       0.88      0.87      0.87      1398
           3       0.89      0.96      0.92      1428
           4       0.80      0.91      0.85      1365
           5       0.91      0.90      0.91      1263
           6       0.87      0.92      0.89      1375
           7       0.96      0.83      0.89      1459
           8       0.97      0.48      0.65      1365
           9       0.92      0.92      0.92      1391
          10       0.71      0.63      0.67      1400
          11       0.91      0.93      0.92      1400
          12       0.64      0.66      0.65      1400
          13       0.75      0.54      0.63      1400
          14       0.66      0.51      0.57      1400
          15       0.43      0.88      0.58      1400
          16       0.46      0.45      0.45      1400
          17       0.94    

In [ ]:
print(classification_report(flatten_y_true,flatten_y_pred)) #RESNET V2

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1381
           1       0.74      0.95      0.83      1575
           2       0.91      0.85      0.88      1398
           3       0.94      0.91      0.93      1428
           4       0.91      0.91      0.91      1365
           5       0.99      0.53      0.70      1263
           6       0.96      0.82      0.88      1375
           7       0.89      0.83      0.86      1459
           8       0.78      0.68      0.73      1365
           9       0.98      0.86      0.92      1391
          10       0.74      0.65      0.69      1400
          11       0.98      0.72      0.83      1400
          12       0.60      0.76      0.67      1400
          13       0.66      0.54      0.60      1400
          14       0.51      0.85      0.64      1400
          15       0.45      0.82      0.58      1400
          16       0.58      0.26      0.36      1400
          17       0.86    

In [ ]:
print(classification_report(flatten_y_true,flatten_y_pred)) #MOBILENet V2

              precision    recall  f1-score   support

           0       0.38      0.99      0.55      1381
           1       0.79      0.98      0.87      1575
           2       0.91      0.96      0.94      1398
           3       0.96      0.95      0.96      1428
           4       0.94      0.98      0.96      1365
           5       0.97      0.90      0.93      1263
           6       0.94      0.94      0.94      1375
           7       0.66      0.97      0.78      1459
           8       0.77      0.97      0.86      1365
           9       0.99      0.95      0.97      1391
          10       0.79      0.60      0.68      1400
          11       0.93      0.93      0.93      1400
          12       0.86      0.33      0.48      1400
          13       0.94      0.45      0.61      1400
          14       0.60      0.46      0.52      1400
          15       0.73      0.68      0.70      1400
          16       0.52      0.17      0.26      1400
          17       0.91    

In [ ]:
mobilenetV2_report = classification_report(flatten_y_true,flatten_y_pred, output_dict=True)
df = pd.DataFrame(mobilenetV2_report).transpose()
df.to_csv('mobilenetV2_report.csv',index=True)

In [ ]:
print(confusion_matrix(flatten_y_true,flatten_y_pred)) #MOBILENet V2

[[1373    3    0    0    1    0    1    0    1    0    0    0    0    0
     0    1    0    1    0    0]
 [   2 1539    0    0    0    1    2    5    0    0    6   13    0    4
     1    1    0    0    0    1]
 [  14    8 1346    1    0    0    0   11   13    0    0    0    0    0
     0    2    0    0    0    3]
 [   9    6    8 1357    0    8    0   18   20    1    0    0    0    0
     0    1    0    0    0    0]
 [   4    8    1    0 1335    0    4    1    0    1    1    1    0    0
     0    1    1    3    1    3]
 [  57    3    0    3    0 1135   40    0    7    3    0    1    0    0
     1   12    0    1    0    0]
 [  63    7    3    0    0    0 1287    0    5    0    0    0    0    0
     0    1    0    8    0    1]
 [  19   11   11    0    3    0    0 1409    2    0    2    2    0    0
     0    0    0    0    0    0]
 [   8    4    1    1    3    0   13    2 1324    0    4    1    0    0
     0    0    0    1    0    3]
 [  27    6    0    0    9    4    0    8    7 1325    

In [ ]:
confusion_matrix(flatten_y_true,flatten_y_pred)

array([[ 351,  420,   28,    0,    0,    1,   11,    1,    0,    0,    0,
           2,   47,   15,   11,   16,   19,   26,   20,  413],
       [   0, 1482,   20,    1,   10,    1,    0,   10,    0,    7,    1,
           1,    0,    1,    0,    0,    0,    0,    0,   41],
       [   0,    3, 1288,    1,    0,    0,    0,    4,    1,    0,    2,
           2,    0,    6,    1,   46,    2,   29,    1,   12],
       [   5,    4,   48, 1045,    0,    8,    2,    8,   84,    7,    1,
           0,    4,   27,    3,   83,    3,   34,   10,   52],
       [   0,    8,    4,    0, 1109,    0,    8,    4,    1,   23,    0,
           6,    4,    0,    7,    8,  104,   41,   29,    9],
       [   2,    4,    0,    6,    0,  995,   11,    0,    4,    0,    2,
           0,    9,    6,    4,  161,    5,    3,    9,   42],
       [   0,   32,    3,    0,    9,    3, 1169,    0,    0,    0,    1,
           0,   26,    8,    8,    4,    1,   70,   13,   28],
       [   3,    6,   14,    3,    2,    